<a class="anchor" id="toc"></a>
# RUN ANOVA TESTS

This notebook works through process of performing ANOVA and various statistical tests on each simulation set.

---
- [WORKSPACE VARIABLES](#workspace-variables)
- [PREPARE DATA](#prepare-data)
- **[TEST ASSUMPTIONS](#test-assumptions)**
- **[RUN ANOVA](#run-anova)**
---

### 1. State the null hypotheses

__Factor A Main Effect__

$H_0: \mu_{1\cdot} = \mu_{2\cdot} =~...~= \mu_{n\cdot}$

There is no difference between or among the factor A row means.

__Factor B Main Effect__

$H_0: \mu_{\cdot 1} = \mu_{\cdot 2} =~...~= \mu_{\cdot m}$

There is no difference between or among the factor B column means.

__Interaction Effect__

$H_0: \mu_{jk} - \mu_{j\cdot} - \mu_{\cdot k} + \mu = 0$ for all $j$, $k$

There is no difference in the factor A by factor B (JK) cell means that cannot be explained by the differences among the factor A (row) means, the factor B (column) means, or both.
In other words, there is no interaction between the two independent variables (factor A and factor B).

### 2. State the alternative hypotheses

__Factor A Main Effect__

$H_A: \mu_{j\cdot} \neq \mu_{k\cdot}$ for some $j$, $k$

The factor A row means differ (are not equal) OR at least one pair or combination of means differ.

__Factor B Main Effect__

$H_A: \mu_{\cdot j} \neq \mu_{\cdot k}$ for some $j$, $k$

The factor B column means differ (are not equal) OR at least one pair or combination of means differ.

__Interaction Effect__

$H_A: \mu_{jk} - \mu_{j\cdot} - \mu_{\cdot k} + \mu \neq 0$ for all $j$, $k$

There are differences along the cell population means that cannot be attributed to main effects.
In other words, there is an interaction between the two independent variables (factor A and factor B).

### 3. Check underlying assumptions

**Check for assumption of independence**

Assess how samples are taken to ensure groups are independent.

**Check for assumption of normality**

_Check for skewness_

$$skewness = \frac{\sqrt{n(n - 1)}}{n - 2}
\cdot \frac{\frac{1}{n}\sum_i^n (y_i - \bar{y})^3}{\left(\frac{1}{n} \sum_i^n (y_i - \bar{y})^2\right)^{3/2}}$$

$$SE(skewness) = \sqrt{\frac{6n(n - 1)}{(n - 2)(n + 1)(n + 3)}}$$

$$standardized~skewness = \frac{skewness}{SE(skewness)}$$

Standardized skewness is approzimately normal for $n > 20$ so we perform a two-tailed test and reject the null hypothesis if the test statistic is beyond the critical z-value for our selected significance level.

> D. N. Joanes and C. A. Gill. Comparing measures of sample skewness and kurtosis. *The Statistician*, 47:183-189, 1998.

_Check Shapiro-Wilks test_

The Shapiro–Wilk test tests the null hypothesis that a sample $x_1, ..., x_n$ came from a normally distributed population.
We can reject this null hypothesis if p value is less than or equal to significance level.

**Check for assumption of homogeneity of variance**

_Check Levene's test_

Levene's test is used to assess the equality of variances for a variable calculated for two or more groups.
It tests the null hypothesis that population variances are equal.
If the resulting p-value is less than some significance level, the obtained differences in sample variances are unlikely to have occurred based on random sampling from a population with equal variances.
Thus, the null hypothesis of equal variances is rejected and it is concluded that there is a difference between the variances in the population.

In the case that variances are different, note that ANOVA is robust to different variances if the sample size is equal.

> MJ Blanca, A Alarcon, J Arnau, R Bono, & R Bendayan. Effect of variance ratio on ANOVA robustness: Might 1.5 be the limit? _Behavior Research Methods_, 50, 937–962 (2018).
>
> _With a variance ratio as large as 9, F-test can, at least for the number of groups and sample sizes considered here, still be used without the Type I error rate being affected by heterogeneity when the design is balanced._

### 4. Calculate main and interaction effects

For a two-way ANOVA, there are two _main effects_ (one for each factor).
An _interaction_ between the two factors occurs if the effect of the level of one factor is not the same across all levels of the other factor (and vice versa).

### 5. Perform additional tests based on effects

__If the interaction IS significant__, we can conclude that both factors affect the outcome and that the effect of changes in one factor depends on the level of the other factor, _i.e._, there is an interaction between the explanatory variables.
The significance results of the main effects are ignored.

We perform Simple Main Effects testing on each factor at each level of the other factor.
The error term is still calculated from the overall data.
To control for Type I error, we use a Bonferroni correction.

__If the interaction IS NOT significant__, then we conclude that an additive model is sufficient, _i.e._, effects of changes in one factor are the same at every fixed level of the other factor.
In other words, the effects on the outcome of a particular level change for one explanatory variable does not depend on the level of the other explanatory variable. 

We re-run the ANOVA without the interaction term and interpret the p-values as in one-way ANOVA. 

### 6. Follow up with ad hoc tests

For significant effects with more than two levels, we can use Tukey multiple comparison to identify which pairs of level are significant.

__If the interaction IS significant__, do comparisons for a given factor at constant levels of the other factor.

__If the interaction IS NOT significant__, do comparisons on full data set. 

<a class="anchor" id="workspace-variables"></a>

### WORKSPACE VARIABLES
<span style="float:right;">[back to top](#toc)</span>

Set up workspace variables for running ANOVA.

- **`PATH`** is the path for analysis files (`.json` and `.csv` files, `.tar.xz` compressed archives)
- **`SIMULATIONS`** is the dictionary of simulations and simulation factors
- **`METRICS`** is the dictionary of metrics and metric scaling

In [ ]:
PATH = "/path/to/analysis/files/"

In [ ]:
SIMULATIONS = {
    "SR": ["dimension", "geometry"],
    "CS": ["volume", "age"],
    "ND": ["profile", "level"],
}

In [ ]:
METRICS = {
    "GROWTH": 30,     # hexagon/day => um/day,
    "SYMMETRY": 1,    # unitless
    "CYCLES": 1/60.0, # minutes => hours
}

<a class="anchor" id="prepare-data"></a>

### PREPARE DATA
<span style="float:right;">[back to top](#toc)</span>

Load and process data for ANOVA.

In [ ]:
from scripts.anova import *

In [ ]:
from scripts.SYSTEM_REPRESENTATION import SYSTEM_REPRESENTATION as SR
from scripts.CELL_STOCHASTICITY import CELL_STOCHASTICITY as CS
from scripts.NUTRIENT_DYNAMICS import NUTRIENT_DYNAMICS as ND

**LOAD DATA**

Load metrics data from the `.SEEDS` (and `.LAYERS.SEEDS` for `SYSTEM_REPRESENTATION`) output files  produced in the basic analysis step.

In [ ]:
data = {}

for metric in METRICS:
    extension = f".SEEDS.{metric}"
    data[f"SR_{metric}"] = SR.get(PATH, load_anova_data, (lambda out: pd.concat(out)), extension, timepoint=15.0)
    data[f"CS_{metric}"] = CS.get(PATH, load_anova_data, (lambda out: pd.concat(out)), extension, timepoint=15.0)
    data[f"ND_{metric}"] = ND.get(PATH, load_anova_data, (lambda out: pd.concat(out)), extension, timepoint=15.0)

**FORMAT DATA**

Format loaded data into single dataframe with consistent columns names.

In [ ]:
columns = ["Y", "X1", "X2", "SIMULATION", "METRIC", "CONTEXT"]
all_dfs = []

for key, values in data.items():
    # Get specific factors and scale for given simulation and metric.
    simulation, metric = key.split("_")
    factors = SIMULATIONS[simulation]
    scale = METRICS[metric]
    
    # Population data frame from loaded data.
    df = pd.DataFrame(columns=columns)

    df["Y"] = values["metric"] * scale
    df["X1"] = values[factors[0]] 
    df["X2"] = values[factors[1]]
    df["CONTEXT"] = values["context"]
    df["SIMULATION"] = simulation
    df["METRIC"] = metric
    
    all_dfs.append(df)
    
all_data = pd.concat(all_dfs)

In [ ]:
all_data[all_data.SIMULATION == "SR"]

<a class="anchor" id="test-assumptions"></a>

### TEST ASSUMPTIONS
<span style="float:right;">[back to top](#toc)</span>

Test ANOVA assumptions.

**GET DATA MEANS**

Calculate the cell and effect means for each simulation set, metric, and context.

In [ ]:
all_cell_means = []
all_effect_means = []

for name, group in all_data.groupby(["SIMULATION", "METRIC", "CONTEXT"]):
    cell_means = calculate_cell_means(group)
    cell_means[["simulation", "metric", "context"]] = list(name)
    all_cell_means.append(cell_means)
    
    effect_means = calculate_effect_means(group)
    effect_means[["simulation", "metric", "context"]] = list(name)
    all_effect_means.append(effect_means)

In [ ]:
cell_means_df = pd.concat(all_cell_means)
cell_means_df.to_csv(f"{PATH}_/ANOVA_cell_means.csv", index=False)

effect_means_df = pd.concat(all_effect_means)
effect_means_df.to_csv(f"{PATH}_/ANOVA_effect_means.csv", index=False)

**TEST NORMALITY**

Check normality using skewness and Shapiro-Wilks for each simulation set, metric, and context.

In [ ]:
for name, group in all_data.groupby(["SIMULATION", "METRIC", "CONTEXT"]):
    print(name)
    skewness = test_skewness(group)
    shapiro = test_shapiro(group)
    print(skewness.to_markdown(index=False))
    print(shapiro.to_markdown(index=False))
    print()

**TEST HOMOGENEITY OF VARIANCE**

Check homogeneity of variance using Levene's for each simulation set, metric, and context.

In [ ]:
for name, group in all_data.groupby(["SIMULATION", "METRIC", "CONTEXT"]):
    print(name)
    levene = test_levene(group)
    print(levene.to_markdown(index=False))
    print()

<a class="anchor" id="run-anova"></a>

### RUN ANOVA
<span style="float:right;">[back to top](#toc)</span>

Run ANOVA and ad hoc tests.

In [ ]:
all_out = {}

for (simulation, metric, context), group in all_data.groupby(["SIMULATION", "METRIC", "CONTEXT"]):
    out = {}
    
    # Run two-way ANOVA with interaction.
    anova = run_two_way_anova_with_interaction(group)
    out["anova_int"] = anova.to_dict()

    # Depending on interaction significance, perform additional tests.
    if anova["is_significant"]["X1:X2"]:
        simple_main_1 = run_simple_main_effects(group, "X1")
        simple_main_2 = run_simple_main_effects(group, "X2")
        tukey_1 = run_tukey_tests(group, "X1", True)
        tukey_2 = run_tukey_tests(group, "X2", True)
        
        out["simple_main"] = {
            "X1": simple_main_1.to_dict(),
            "X2": simple_main_2.to_dict(),
        }
        
        out["tukey"] = {
            "X1": tukey_1.to_dict("records") if tukey_1 is not None else [],
            "X2": tukey_2.to_dict("records") if tukey_2 is not None else [],
        }
    else:
        anova_noint = run_two_way_anova_without_interaction(group)
        tukey_1 = run_tukey_tests(group, "X1", False)
        tukey_2 = run_tukey_tests(group, "X2", False)
        
        out["anova_noint"] =  anova_noint.to_dict()
        
        out["tukey"] = {
            "X1": tukey_1.to_dict("records") if tukey_1 is not None else [],
            "X2": tukey_2.to_dict("records") if tukey_2 is not None else [],
        }
    
    all_out[f"{simulation}_{metric}_{context}"] = out

with open(f"{PATH}_/ANOVA_testing.json", "w") as f:
    jn = json.dumps(all_out, indent = 4, separators = (',', ':'), sort_keys=True)
    f.write(jn.replace("NaN", '"nan"'))